In [1]:
import robotic as ry
import numpy as np
import time


C = ry.Config()
C.addFile("cross_map.g")

r1 = "r1"
r2 = "r2"
m1 = "m1"
m2 = "m2"
g1 = "g1"
g2 = "g2"

robots = [r1, r2]
m_objects = [m1, m2]
goals = [g1, g2]

xy_dim = np.array([[1, 0, 0], [0, 1, 0]])


def komo_solver(robots, m_objects, goals):
    
    if not isinstance(robots, list):
        robots = [robots]
    if not isinstance(m_objects, list):
        m_objects = [m_objects]
    if not isinstance(goals, list):
        goals = [goals]
    
    komo = ry.KOMO(C, phases=2, slicesPerPhase=10, kOrder=2, enableCollisions=True)  
    komo.addControlObjective([], 0)
    komo.addControlObjective([], 2)
    
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)     
    
    for i in range(len(robots)):
        komo.addObjective([1, -1], ry.FS.distance, [robots[i], m_objects[i]], ry.OT.eq) 
        komo.addModeSwitch([1,-1], ry.SY.stable, [robots[i], m_objects[i]], True) 
        komo.addObjective([2] , ry.FS.positionDiff, [m_objects[i], goals[i]], ry.OT.eq) 

    ret = ry.NLP_Solver(komo.nlp(), verbose=4).solve() 
    
    return (komo, ret)


def collisions(config, path):
    
    counter = 0
    for state in path:
        if counter == 10:
            config.attach(m1, r1)
        config.setJointState(state)
        config.view()
        config.computeCollisions()
        y, j = config.eval(ry.FS.accumulatedCollisions)
        print(y)
        counter += 1
        #time.sleep(1)
        

In [2]:
# try to solve situation for both case 
#komo_both, ret_both = komo_solver(robots, m_objects, goals)
komo_both, ret_both = komo_solver(r1, m_objects, goals)


print(f"Both objects simultaneously\neq: {ret_both.eq}\nSos: {ret_both.sos}\nisFeasible: {ret_both.feasible}")




if ret_both.feasible == True:
    path = komo_both.getPath()    
    collisions(C, path)
    
    
    
else:
    pass
    



Both objects simultaneously
eq: 0.03090137447258578
Sos: 510.618475101836
isFeasible: True
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
STACK9 /bin/python3(_PyEval_EvalFrameDefault
STACK8 /bin/python3(_PyFunction_Vectorcall
STACK7 /bin/python3(_PyEval_EvalFrameDefault
STACK6 /bin/python3(+0x198a6b) [0x558f126faa6b]
STACK5 /bin/python3(_PyObject_MakeTpCall
STACK4 /bin/python3(+0x18ae12) [0x558f126ece12]
STACK3 /home/alperen/.local/lib/python3.10/site-packages/robotic/_robotic.so(+0x3f21e) [0x7fb43462021e]
STACK2 /home/alperen/.local/lib/python3.10/site-packages/robotic/_robotic.so(+0x81a3d) [0x7fb434662a3d]
STACK1 rai::Configuration::setJointState(rai::ArrayDouble const&)
STACK0 rai::LogToken::~LogToken()


== ERROR:kin.cpp:setJointState:675(-2) CHECK_EQ failed: '_q.N'=6 'N'=3 -- wrong joint state dimensionalities


RuntimeError: kin.cpp:setJointState:675(-2) CHECK_EQ failed: '_q.N'=6 'N'=3 -- wrong joint state dimensionalities

In [ ]:
komo_both.view_play()